In [1]:
!pip  install sk-video

In [2]:
import keras.backend as K
from keras.models import Sequential, Model
from keras.utils.data_utils import get_file
from keras.layers import Input, Conv3D, MaxPooling3D, Dense, Flatten, ZeroPadding3D, Dropout, Subtract
import skvideo.io
import numpy as np

Using TensorFlow backend.


In [0]:
WEIGHTS_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/sports1M_weights_tf.h5'
C3D_MEAN_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/c3d_mean.npy'

In [0]:
if K.image_data_format() == 'channels_last':    
    shape0 = (16,112,112,3)    
else:   
    shape0 = (3,16,112,112)

model_base = Sequential()
    
model_base.add(Conv3D(64, 3, activation='relu', padding='same', name='conv1', input_shape=shape0))
model_base.add(MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2), padding='same', name='pool1'))
    
model_base.add(Conv3D(128, 3, activation='relu', padding='same', name='conv2'))
model_base.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool2'))
    
model_base.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3a'))
model_base.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3b'))
model_base.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool3'))
    
model_base.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4a'))
model_base.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4b'))
model_base.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool4'))
    
model_base.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5a'))
model_base.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5b'))
model_base.add(ZeroPadding3D(padding=(0,1,1)))
model_base.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool5'))
    
model_base.add(Flatten())
    
model_base.add(Dense(4096, activation='relu', name='fc6'))
model_base.add(Dropout(0.5))
model_base.add(Dense(4096, activation='relu', name='fc7'))
model_base.add(Dropout(0.5))
model_base.add(Dense(487, activation='softmax', name='fc8'))

weights_path = get_file('sports1M_weights_tf.h5',
                         WEIGHTS_PATH,
                         cache_subdir='models',
                         md5_hash='b7a93b2f9156ccbebe3ca24b41fc5402')
        
model_base.load_weights(weights_path)

model_base.pop()




In [5]:
start_window = Input(shape=shape0, dtype='float32', name='start_window')
followup_window = Input(shape=shape0, dtype='float32', name='followup_window')

fc7 =model_base(start_window)

drop2 = Dropout(0.5)(fc7)
fc8 = Dense(2, activation='sigmoid', name='fc8')(drop2)


out1 = fc7
out2 = model_base(followup_window)

out = Subtract(name='out')([out1, out2])

model = Model([start_window,followup_window],[fc8,out])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
start_window (InputLayer)       (None, 16, 112, 112, 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         77995776    start_window[0][0]               
                                                                 followup_window[0][0]            
__________________________________________________________________________________________________
followup_window (InputLayer)    (None, 16, 112, 112, 0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 4096)         0           sequential_1[1][0]               
__________

In [0]:
def loss_classification(y_true, y_pred):
    return -K.mean(K.log(K.dot(y_pred,K.transpose(y_true))), axis=-1)
    
def loss_temporal_consistency(y_true,y_pred):
    return K.mean(K.square(K.dot(out,K.transpose(y_true))), axis=-1)

In [0]:
model.compile(optimizer='adam',
              loss={'fc8': loss_classification, 'out': loss_temporal_consistency},
              loss_weights={'fc8': 1., 'out': 1})

In [8]:
import matplotlib.pyplot as plt 


path1 = '../content/Normal_Videos_015_x264.mp4'
path2 = '../content/Normal_Videos_050_x264.mp4'




Video = skvideo.io.vread(path1,
                          outputdict={
                            "-sws_flags": "bilinear",
                            "-s": "112x112"
                          })

sequence1 = Video[:16]
sequence2 = Video[16:32]

sequence1  = np.expand_dims(sequence1, axis=0)
sequence2 = np.expand_dims(sequence2, axis=0)

features = model.predict([sequence1,sequence2])
features

[array([[0.46909508, 0.3023162 ]], dtype=float32),
 array([[0.45791927, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]], dtype=float32)]